# Setup

In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

In [ ]:
filtered_d0 <- readRDS('../Seurat_Objects/filtered_d0_label_transfer.rds')

## Add Metadata

### Add Pediatric vs Senior Designation

In [ ]:
filtered_d0 <- SetIdent(filtered_d0, value = 'pbmc_sample_id')
filtered_d0 <- RenameIdents(filtered_d0, 
                            'PB00593-04'='Senior', 'PB00323-02'='Senior', 'PB00368-04'='Senior', 'PB00334-03'='Senior',
                            'PB00377-03'='Senior', 'PB00545-02'='Senior', 'PB00599-02'='Senior', 'PB00353-03'='Senior',
                            'PB00124-02'='Pediatric','PB00172-02'='Pediatric','PB00182-02'='Pediatric','PB00127-02'='Pediatric',
                            'PB00807-02'='Pediatric','PB00173-02'='Pediatric','PB00192-02'='Pediatric','PB00197-02'='Pediatric')
                            
filtered_d0$pediatric_senior <- Idents(filtered_d0)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 5)
DimPlot(filtered_d0, group.by = 'pediatric_senior', order = c('Pediatric','Senior'))

### Add 10x CCR7 Imputation (among others)

In [ ]:
t_sub_10x_ref <- readRDS(file = '../Seurat_Objects/10x_5k_T_cell_subset.rds')

In [ ]:
DefaultAssay(t_sub_10x_ref) <- 'SCT'
DefaultAssay(filtered_d0) <- 'SCT'
anchors <- FindTransferAnchors(
reference = t_sub_10x_ref,
query = filtered_d0,
normalization.method = "SCT",
dims = 1:30
)

In [ ]:
filtered_d0 <- TransferData(anchorset = anchors, reference = t_sub_10x_ref, query = filtered_d0,
                refdata = list(x10_ADT = 'ADT'))

In [ ]:
filtered_d0 <- NormalizeData(filtered_d0, assay = 'ADT', normalization.method = 'CLR', margin = 2)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)
FeaturePlot(filtered_d0, features = c('x10ADT_CD197','adt_CD197'), reduction = 'umap',
           min.cutoff = 'q03', max.cutoff = 'q97') & scale_color_viridis()

## Split dataset by batches 

In [ ]:
filtered_d0 <- SetIdent(filtered_d0, value = 'batch_id')
table(filtered_d0$batch_id)

In [ ]:
b065 <- subset(filtered_d0, idents = 'B065')
b069 <- subset(filtered_d0, idents = 'B069')
b076 <- subset(filtered_d0, idents = 'B076')

In [ ]:
b065
b069
b076

In [ ]:
b065 <- NormalizeData(b065, normalization.method = 'CLR', margin = 2, assay = 'ADT')
b069 <- NormalizeData(b069, normalization.method = 'CLR', margin = 2, assay = 'ADT')
b076 <- NormalizeData(b076, normalization.method = 'CLR', margin = 2, assay = 'ADT')

## Create major population subsets from each batch

In [ ]:
b065_b069_cutoffs=data.frame(x1=c(0,2.6,0,2.6), x2=c(2.6,6,2.6,6), y1=c(0,0,3,3), y2=c(3,3,6.5,6.5), Pops=c('dnT','CD4','CD8','dpT'))
b076_cutoffs=data.frame(x1=c(0,2.1,0,2.4), x2=c(2.1,5,2.4,5), y1=c(0,0,2.8,2.8), y2=c(2.8,2.8,6.5,6.5), Pops=c('dnT','CD4','CD8','dpT'))

In [ ]:
b065_adt_mtx_data <- as.data.frame(b065@assays$ADT@data)
b065_adt_mtx_data <- t(b065_adt_mtx_data)
b065_adt_mtx_data <- as.data.frame(b065_adt_mtx_data)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)
ggplot() + 
geom_point(b065_adt_mtx_data, mapping = aes(x = CD4, y = CD8a)) + 
geom_rect(data=b065_b069_cutoffs, mapping=aes(xmin=x1, xmax=x2, ymin=y1, ymax=y2, fill = Pops), color="black", alpha = 0.1) +
ggtitle("B065") + theme(plot.title = element_text(size = 30))

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
ggplot() + 
geom_point(b065_adt_mtx_data, mapping = aes(x = CD4, y = CD8a), size = 0.5) + 
# geom_rect(data=b065_b069_cutoffs, mapping=aes(xmin=x1, xmax=x2, ymin=y1, ymax=y2), color="black", alpha = 0.1) +
ggtitle("B065") + theme(plot.title = element_text(size = 30))

In [ ]:
#b065_b069_cutoffs=data.frame(x1=c(0,2.6,0,2.6), x2=c(2.6,6,2.6,6), y1=c(0,0,3,3), y2=c(3,3,6.5,6.5), Pops=c('dnT','CD4','CD8','dpT'))
b065_cd8_subset <- subset(b065, subset = adt_CD8a > 3 & adt_CD4 < 2.6)
b065_cd4_subset <- subset(b065, subset = adt_CD8a < 3 & adt_CD4 > 2.6)
b065_dpT_subset <- subset(b065, subset = adt_CD8a > 3 & adt_CD4 > 2.6)
b065_dnT_subset <- subset(b065, subset = adt_CD8a < 3 & adt_CD4 < 2.6)

In [ ]:
b069_adt_mtx_data <- as.data.frame(b069@assays$ADT@data)
b069_adt_mtx_data <- t(b069_adt_mtx_data)
b069_adt_mtx_data <- as.data.frame(b069_adt_mtx_data)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)
ggplot() + 
geom_point(b069_adt_mtx_data, mapping = aes(x = CD4, y = CD8a)) + 
geom_rect(data=b065_b069_cutoffs, mapping=aes(xmin=x1, xmax=x2, ymin=y1, ymax=y2, fill = Pops), color="black", alpha = 0.1) +
ggtitle("B069") + theme(plot.title = element_text(size = 30))

In [ ]:
#b065_b069_cutoffs=data.frame(x1=c(0,2.6,0,2.6), x2=c(2.6,6,2.6,6), y1=c(0,0,3,3), y2=c(3,3,6.5,6.5), Pops=c('dnT','CD4','CD8','dpT'))
b069_cd8_subset <- subset(b069, subset = adt_CD8a > 3 & adt_CD4 < 2.6)
b069_cd4_subset <- subset(b069, subset = adt_CD8a < 3 & adt_CD4 > 2.6)
b069_dpT_subset <- subset(b069, subset = adt_CD8a > 3 & adt_CD4 > 2.6)
b069_dnT_subset <- subset(b069, subset = adt_CD8a < 3 & adt_CD4 < 2.6)

In [ ]:
b076_adt_mtx_data <- as.data.frame(b076@assays$ADT@data)
b076_adt_mtx_data <- t(b076_adt_mtx_data)
b076_adt_mtx_data <- as.data.frame(b076_adt_mtx_data)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)
ggplot() + 
geom_point(b076_adt_mtx_data, mapping = aes(x = CD4, y = CD8a)) + 
geom_rect(data=b076_cutoffs, mapping=aes(xmin=x1, xmax=x2, ymin=y1, ymax=y2, fill = Pops), color="black", alpha = 0.1) +
ggtitle("B076") + theme(plot.title = element_text(size = 30))

In [ ]:
#b076_cutoffs=data.frame(x1=c(0,2.1,0,2.4), x2=c(2.1,5,2.4,5), y1=c(0,0,2.8,2.8), y2=c(2.8,2.8,6.5,6.5), Pops=c('dnT','CD4','CD8','dpT'))
b076_cd8_subset <- subset(b076, subset = adt_CD8a > 2.8 & adt_CD4 < 2.4)
b076_cd4_subset <- subset(b076, subset = adt_CD8a < 2.8 & adt_CD4 > 2.1)
b076_dpT_subset <- subset(b076, subset = adt_CD8a > 2.8 & adt_CD4 > 2.4)
b076_dnT_subset <- subset(b076, subset = adt_CD8a < 2.8 & adt_CD4 < 2.1)

## Combine Major Populations

In [ ]:
cd8_subset <- merge(b065_cd8_subset, c(b069_cd8_subset, b076_cd8_subset))
cd4_subset <- merge(b065_cd4_subset, c(b069_cd4_subset, b076_cd4_subset))
dpT_subset <- merge(b065_dpT_subset, c(b069_dpT_subset, b076_dpT_subset))
dnT_subset <- merge(b065_dnT_subset, c(b069_dnT_subset, b076_dnT_subset))

### Add metadata for level 0

In [ ]:
cd8_subset$celltype_l0 <- rep('CD8', length(colnames(cd8_subset@assays$RNA)))
cd4_subset$celltype_l0 <- rep('CD4', length(colnames(cd4_subset@assays$RNA)))
dpT_subset$celltype_l0 <- rep('dpT', length(colnames(dpT_subset@assays$RNA)))
dnT_subset$celltype_l0 <- rep('dnT', length(colnames(dnT_subset@assays$RNA)))

# Save Objects

In [ ]:
saveRDS(cd8_subset, file = '../Seurat_Objects/cd8_subset.rds')

In [ ]:
saveRDS(cd4_subset, file = '../Seurat_Objects/cd4_subset.rds')

In [ ]:
saveRDS(dpT_subset, file = '../Seurat_Objects/dpT_subset.rds')

In [ ]:
saveRDS(dnT_subset, file = '../Seurat_Objects/dnT_subset.rds')

In [ ]:
sessionInfo()